In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as plt_common_utils
import shutil
import os

In [7]:
jobs = ["depth_fit", "rf", "merge_dataframes", "merge_dataframes_recording", "plot_basicvis", "plot_rf"]
to_do = [
    ("gaussian_2d", None, 1),
    ("gaussian_2d", "even", 1),
    ("gaussian_additive", None, 1),
    ("gaussian_OF", None, 1),
    ("gaussian_2d", None, 5),
    ("gaussian_additive", None, 5),
    ("gaussian_OF", None, 5),
    ("gaussian_ratio", None, 1),
    ("gaussian_ratio", None, 5),
]

to_do_recording = [
    ("gaussian_2d", None, 1),
    ("gaussian_2d", None, 5),
]

jobs_recording=[]

for model, trials, k_folds in to_do:
    name = f"{model}"
    if trials is not None:
        name += "_crossval"
    name += f"_k{k_folds}"
    jobs.append(f"rsof_fit_{name}")

for model, trials, k_folds in to_do_recording:
    name = f"{model}"
    if trials is not None:
        name += "_crossval"
    name += f"_k{k_folds}"
    jobs_recording.append(f"rsof_fit_{name}_recording")
    
jobs, jobs_recording
    


(['depth_fit',
  'rf',
  'merge_dataframes',
  'merge_dataframes_recording',
  'plot_basicvis',
  'plot_rf',
  'rsof_fit_gaussian_2d_k1',
  'rsof_fit_gaussian_2d_crossval_k1',
  'rsof_fit_gaussian_additive_k1',
  'rsof_fit_gaussian_OF_k1',
  'rsof_fit_gaussian_2d_k5',
  'rsof_fit_gaussian_additive_k5',
  'rsof_fit_gaussian_OF_k5',
  'rsof_fit_gaussian_ratio_k1',
  'rsof_fit_gaussian_ratio_k5'],
 ['rsof_fit_gaussian_2d_k1_recording', 'rsof_fit_gaussian_2d_k5_recording'])

In [28]:
SESSION_LIST = [    
    "PZAH6.4b_S20220419",
    "PZAH6.4b_S20220421",
    "PZAH6.4b_S20220426",
    "PZAH6.4b_S20220428",
    "PZAH6.4b_S20220429",
    "PZAH6.4b_S20220503",
    "PZAH6.4b_S20220505",
    "PZAH6.4b_S20220506",
    "PZAH6.4b_S20220510",
    "PZAH6.4b_S20220511",
    "PZAH6.4b_S20220512",
    "PZAH6.4b_S20220516",
    "PZAH6.4b_S20220517",
    "PZAH6.4b_S20220519",
    "PZAH6.4b_S20220524",
    "PZAH6.4b_S20220526",

    "PZAG3.4f_S20220419",
    "PZAG3.4f_S20220421",
    "PZAG3.4f_S20220422",
    "PZAG3.4f_S20220426",
    "PZAG3.4f_S20220429",
    "PZAG3.4f_S20220503",
    "PZAG3.4f_S20220504",
    "PZAG3.4f_S20220505",
    "PZAG3.4f_S20220508",
    "PZAG3.4f_S20220509",
    "PZAG3.4f_S20220510",
    "PZAG3.4f_S20220511",
    "PZAG3.4f_S20220512",
    "PZAG3.4f_S20220517",
    "PZAG3.4f_S20220519",
    "PZAG3.4f_S20220520",
    "PZAG3.4f_S20220523",
    "PZAG3.4f_S20220524",
    "PZAG3.4f_S20220526",
    "PZAG3.4f_S20220527",
    
    "PZAH8.2h_S20221208",
    "PZAH8.2h_S20221213",
    "PZAH8.2h_S20221215",
    "PZAH8.2h_S20221216",
    "PZAH8.2h_S20230113", # need more memory for RF fit
    "PZAH8.2h_S20230116", # need more memory for RF fit
    "PZAH8.2h_S20230126", # imaging trigger needs to be processed
    "PZAH8.2h_S20230127", # need more memory for RF fit
    "PZAH8.2h_S20230202",
    "PZAH8.2h_S20230224",
    "PZAH8.2h_S20230302",
    "PZAH8.2h_S20230303",
    "PZAH8.2h_S20230314",
    "PZAH8.2h_S20230321",

    "PZAH8.2i_S20231208",
    "PZAH8.2i_S20231209",
    "PZAH8.2i_S20231213",
    "PZAH8.2i_S20231215",
    "PZAH8.2i_S20230110",
    "PZAH8.2i_S20230116",
    "PZAH8.2i_S20230117",
    "PZAH8.2i_S20230127",
    "PZAH8.2i_S20230203",
    "PZAH8.2i_S20230209",
    "PZAH8.2i_S20230216",
    "PZAH8.2i_S20230220",
    "PZAH8.2i_S20230324",
    "PZAH8.2i_S20230330",
    "PZAH8.2i_S20230404",
        
    "PZAH8.2f_S20230109",
    "PZAH8.2f_S20230117",
    "PZAH8.2f_S20230126",  # need more memory for RF
    "PZAH8.2f_S20230131",
    "PZAH8.2f_S20230202",
    "PZAH8.2f_S20230206",
    "PZAH8.2f_S20230214",
    "PZAH8.2f_S20230223",
    "PZAH8.2f_S20230313", # need more memory for RF
    
    "PZAH10.2d_S20230526",
    "PZAH10.2d_S20230531",
    "PZAH10.2d_S20230602",
    "PZAH10.2d_S20230608",
    "PZAH10.2d_S20230613",
    "PZAH10.2d_S20230615",
    "PZAH10.2d_S20230616",
    "PZAH10.2d_S20230623",
    "PZAH10.2d_S20230626",
    "PZAH10.2d_S20230627",
    "PZAH10.2d_S20230628",
    "PZAH10.2d_S20230703",
    "PZAH10.2d_S20230706",
    "PZAH10.2d_S20230704", # depth tuning fit needs more iteration
    "PZAH10.2d_S20230707",
    "PZAH10.2d_S20230725",
    "PZAH10.2d_S20230728",
    "PZAH10.2d_S20230818",
    "PZAH10.2d_S20230821",
    "PZAH10.2d_S20230920",
    "PZAH10.2d_S20230922",


    "PZAH10.2f_S20230525",
    "PZAH10.2f_S20230601",
    "PZAH10.2f_S20230605",
    "PZAH10.2f_S20230606",
    "PZAH10.2f_S20230609",
    "PZAH10.2f_S20230614",
    "PZAH10.2f_S20230615",
    "PZAH10.2f_S20230616",
    "PZAH10.2f_S20230622",
    "PZAH10.2f_S20230623",
    "PZAH10.2f_S20230626",
    "PZAH10.2f_S20230627",
    "PZAH10.2f_S20230703",
    "PZAH10.2f_S20230705",
    "PZAH10.2f_S20230706",
    "PZAH10.2f_S20230707", 
    "PZAH10.2f_S20230727", # need longer time for RF fit
    "PZAH10.2f_S20230801",
    "PZAH10.2f_S20230804",
    "PZAH10.2f_S20230807", 
    "PZAH10.2f_S20230808",
    "PZAH10.2f_S20230811",
    "PZAH10.2f_S20230817",
    "PZAH10.2f_S20230822",
    "PZAH10.2f_S20230908",
    "PZAH10.2f_S20230912",
    "PZAH10.2f_S20230914",
    "PZAH10.2f_S20230924",
    ]

In [1]:
col_list = [
    "roi",
    "best_depth",
    "preferred_depth_closedloop",
    "preferred_depth_closedloop_running",
    "preferred_depth_closedloop_notrunning",
    "preferred_depth_closedloop_crossval",
    "depth_tuning_popt_closedloop",
    "depth_tuning_test_rsq_closedloop",
    "depth_tuning_test_spearmanr_pval_closedloop",
    "depth_tuning_test_spearmanr_rval_closedloop",
    
    "rf_coef_closedloop",
    "rf_coef_ipsi_closedloop",
    "rf_rsq_ipsi_closedloop",
    "rf_reg_xy_closedloop",
    
    "preferred_RS_closedloop_g2d",
    "preferred_RS_closedloop_crossval_g2d",
    "preferred_OF_closedloop_g2d",
    "preferred_OF_closedloop_crossval_g2d",
    "preferred_RS_closedloop_gadd",
    "preferred_OF_closedloop_gof",
    "preferred_RSOFratio_closedloop_gratio",
    
    "rsof_test_rsq_closedloop_g2d",
    "rsof_rsq_closedloop_g2d",
    "rsof_popt_closedloop_g2d",
    "rsof_test_rsq_closedloop_gadd",
    "rsof_test_rsq_closedloop_gof",
    "rsof_test_rsq_closedloop_gratio",
    
]
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
for session in ["PZAH6.4b_S20220503"]:
    print(f"---{session}---")
    neurons_ds = pipeline_utils.create_neurons_ds(
        session_name=session_name,
        flexilims_session=flexilims_session,
        project=None,
        conflicts="skip",
    )
    if not os.path.exists(neurons_ds.path_full):
        print("No neurons_df found")
    else:
        neurons_df = pd.read_pickle(neurons_ds.path_full)
        for col in col_list:
            if col not in neurons_df.columns:
                print(f"Missing{col}")

---PZAH6.4b_S20220503---


NameError: name 'pipeline_utils' is not defined

In [38]:
JOBS = ["depth_fit", "rf", "merge_dataframes", "merge_dataframes_recording", "plot_basicvis", "plot_rf"]
to_do = [
    ("gaussian_2d", None, 1),
    ("gaussian_2d", "even", 1),
    ("gaussian_additive", None, 1),
    ("gaussian_OF", None, 1),
    ("gaussian_2d", None, 5),
    ("gaussian_additive", None, 5),
    ("gaussian_OF", None, 5),
    ("gaussian_ratio", None, 1),
    ("gaussian_ratio", None, 5),
]

to_do_recording = [
    ("gaussian_2d", None, 1),
    ("gaussian_2d", None, 5),
]

JOBS_RECORDING=[]

for model, trials, k_folds in to_do:
    name = f"{model}"
    if trials is not None:
        name += "_crossval"
    name += f"_k{k_folds}"
    JOBS.append(f"rsof_fit_{name}")

for model, trials, k_folds in to_do_recording:
    name = f"{model}"
    if trials is not None:
        name += "_crossval"
    name += f"_k{k_folds}"
    JOBS_RECORDING.append(f"rsof_fit_{name}_recording")

latest_day = 20240612

for session_name in ["PZAH6.4b_S20220419"]:
    print(f"---{session_name}---")
    neurons_ds = pipeline_utils.create_neurons_ds(
        session_name=session_name,
        flexilims_session=flexilims_session,
        project=None,
        conflicts="skip",
    )
    if not os.path.exists(neurons_ds.path_full.parent/"finished.pickle"):
        print("Not processed")
    else:
        finished = pd.read_pickle(neurons_ds.path_full.parent/"finished.pickle")
        for job in JOBS:
            if f"{job}_finished_day" not in finished.columns:
                print(f"Missing {job}")
            else:
                if finished[f"{job}_finished_day"].values.astype("int")[0] < latest_day:
                    print(f"Outdated {job}, last finished {finished[f'{job}_finished_day'].values.astype('int')[0]}")
                    
        jobs_recording_names = [col for col in finished.columns if any(sub in col for sub in JOBS_RECORDING)]
        for job in jobs_recording_names:
            job = "_".join(job.split("_")[:-2])
            if f"{job}_finished_day" not in finished.columns:
                print(f"Missing {job}")
            else:
                if finished[f"{job}_finished_day"].values.astype("int")[0] < latest_day:
                    print(f"Outdated {job}, last finished {finished[f'{job}_finished_day'].values.astype('int')[0]}")

---PZAH6.4b_S20220419---
Missing merge_dataframes
Missing merge_dataframes_recording
Missing plot_basicvis
Missing plot_rf
Missing rsof_fit_gaussian_2d_k1
Missing rsof_fit_gaussian_2d_crossval_k1
Missing rsof_fit_gaussian_additive_k1
Missing rsof_fit_gaussian_OF_k1
Missing rsof_fit_gaussian_2d_k5
Missing rsof_fit_gaussian_OF_k5
Missing rsof_fit_gaussian_ratio_k1
Missing rsof_fit_gaussian_ratio_k5
Missing rsof_fit_gaussian_2d_k1_recording_R145152_SpheresPermTubeReward_closedloop_0
Missing rsof_fit_gaussian_2d_k1_recording_R145152_SpheresPermTubeReward_closedloop_0
Missing rsof_fit_gaussian_2d_k5_recording_R145152_SpheresPermTubeReward_closedloop_0
Missing rsof_fit_gaussian_2d_k5_recording_R145152_SpheresPermTubeReward_closedloop_0
